In [1]:
import sys
import purly

To set things up we'll need to stand up our model server. By default the server has a max refresh rate of `25` hertz. However for this example to be convincingly smooth we'll want to bump that up to about `60`. If you want to unlock the refresh rate set `refresh=None`. We'll then hook up a layout object to a model resource (see the introductory example  or [read the docs](https://github.com/rmorshea/purly#purly) if this doesn't make sense).

In [2]:
from example_utils import localhost

# increase model server refresh cap to 60 hertz.
purly.state.Machine(refresh=60).daemon()

# name the layout resource "simple-slider" and connect to the update stream
websocket_url = localhost('ws', 8000) + '/model/simple-slider/stream'
layout = purly.Layout(websocket_url)

[2018-08-16 22:50:17 -0700] [7239] [INFO] Goin' Fast @ http://127.0.0.1:8000
[2018-08-16 22:50:17 -0700] [7239] [INFO] Starting worker [7239]
[2018-08-16 22:50:18 -0700] - (sanic.access)[INFO][1:2]: GET http://127.0.0.1:8000/model/simple-slider/assets/index.html  200 573
[2018-08-16 22:50:18 -0700] - (sanic.access)[INFO][1:2]: GET http://127.0.0.1:8000/model/simple-slider/assets/static/css/main.785d0ef8.css  200 118
[2018-08-16 22:50:18 -0700] - (sanic.access)[INFO][1:2]: GET http://127.0.0.1:8000/model/simple-slider/assets/static/js/main.6da5cf05.js  200 192350
[2018-08-16 22:50:23 -0700] [7239] [INFO] KeepAlive Timeout. Closing connection.
[2018-08-16 22:50:23 -0700] [7239] [INFO] KeepAlive Timeout. Closing connection.
[2018-08-16 22:50:30 -0700] [7239] [INFO] Stopping worker [7239]
[2018-08-16 22:50:50 -0700] [7239] [INFO] Server Stopped


We'll then create a simple slider with values between 1 and 10 and add it to the layout before displaying.

In [3]:
slider = layout.html('input', type='range', value="5", min=1, max=10, step=1)
layout.children.append(slider)
layout.sync()
layout

You should now see the slider in the output above!

However when you try to move it, nothing happens, so we'll need to hook into some mouse events to make the display animate. To do that requires the `onChange` event which can be captured via the `on` decorator of the `slider` element. The one detail that significant is passing the string `'value'` as a second argument to the decorator. This will sync the slider's value between Python and the browser when the event occurs.

Once we've done that we can simply print out the now synced `slider.attributes['value']` as it changes.

In [4]:
@slider.on('Change', 'value')
def printer():
    sys.stdout.write('\r%02d' % int(slider.attributes['value']))
    sys.stdout.flush()

In [5]:
layout.serve()

01

KeyboardInterrupt: 